# Supernovae Lab 1

 Goal: In this first lab, we will explore how to:
 - determine the population of binaries that reach the end of their lives,

 - identify their collapse mechanisms (electron capture vs. iron core-collapse),

 - link them to the observed supernova types they produce,

 - calculate their ejecta masses

 - examine their main formation channels using also their van den Heuvel diagrams.

Our analysis will be based on post-processing an already completed POSYDON population run.

# 1. Managing Large Binary Populations and Identifying Stellar End-of-Life Stages



<span style="font-size:18px"> In your own research with POSYDON you will likely work with much larger populations than the small examples we have been running here. This is crucial for obtaining realistic statistical results for the populations you want to study. In this session, we will practice handling such large populations. To begin, we will review the columns stored in the POSYDON population output (saved in .h5 files) and examine the evolution of a specific binary system </span>

## Loading Your Population

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from posydon.config import PATH_TO_POSYDON_DATA

import os
import posydon
import pandas as pd

data_path = os.path.join(os.path.dirname(PATH_TO_POSYDON_DATA), "2025_school_data/populations/5K_pops")
pop_name = '1e+00_Zsun_population.h5'

path_to_pop_full = os.path.join(data_path, pop_name)

For more info: https://posydon.org/POSYDON/latest/api_reference/posydon.popsyn.html#module-posydon.popsyn.synthetic_population

In [ ]:
from posydon.popsyn.synthetic_population import Population
pop = Population(path_to_pop_full, chunksize=1000) 

## Printing out the population's initial parameters to see the settings used to create the file.

In [ ]:
# the parmaeter space and initial conditions of the run population
pop.ini_params

## Printing out the columns saved in the history data frame of the population

In [ ]:
# columns of the parameters of each POSYDON binary object
print(pop.history.columns)

## Examine the evolution of a specific system from your population, focusing on the columns of interest.

In [ ]:
# Let's see the evolution of one system
col = ['step_names','time','state','event','S1_state','S2_state','S1_mass','S2_mass','orbital_period','separation','S1_surface_h1','S1_surface_n14','S2_surface_h1','S2_surface_n14']
pop.history[3][col]

Above, you can see the evolution of a specific binary system. In this lab, we will focus on the final stages of stellar evolution by identifying the rows and columns that provide information about a star’s end of life.

In POSYDON, the end stages are marked in the `event` column with either `CC1` or `CC2`, indicating whether the collapse corresponds to the primary or secondary star. Immediately after an `event = CC1`, you will notice that the `step_names` column transitions to `step_SN`. Our first task is to establish how to systematically identify all CC1 and CC2 events across the population.

POSYDON distinguishes between two types of core-collapse events: `CC1` for primaries (star_1, the initially more massive star) and `CC2` for secondaries (star_2). For single stars, only `CC1` events occur, since they are treated as primaries. With this in mind, let’s find all the `CC1` and `CC2` events that arise from the different evolutionary channels.

**Disclaimer**: Please note that for 'historical reasons' in POSYDON, `CC1` and `CC2` is kept for the final / end point (i.e. if possible reaching carbon-core-depletion for massive stars) of all stars. Indepedent of the outcome if it is `NS`, a `BH`, but even a `WD`, or `massless remnant` (in the case of PISN) all system that finish their evolution, pass through the `CC1` or `CC2` events. 

# Analyzing a population of SNe with `create_transient_population` function

To extract only the relevant rows from a large set of (binary) stellar systems during post-processing, we can use the `create_transient_population` function. This allows us to retain just the events and information of interest. In practice, this means transforming a dataset that tracks binary systems and their full evolution into a more focused dataset containing only `CC1` and `CC2` events, that is, a dataset of supernovae (SNe).

For more details, see the documentation: https://posydon.org/POSYDON/latest/components-overview/pop_syn/synthetic_population.html#creating-a-transientpopulation

### `create_transient_population(func, transient_name, oneline_cols=None, hist_cols=None)`

Create a **TransientPopulation** using a custom function.

This method constructs a transient population by applying the provided function to chunks of the population data (history, oneline, and formation channels).  

The results are stored as a pandas DataFrame at  
`'/transients/{transient_name}'` in the population file.

Processing is chunked to avoid memory issues. Performance can be improved by limiting `oneline_cols` and `hist_cols` to only the columns required.

---

#### Parameters
- **func** : `function`  
  Function to apply to the parsed population.  
  Must accept **three arguments**:
  - `history_chunk : pd.DataFrame`  
  - `oneline_chunk : pd.DataFrame`  
  - `formation_channels_chunk : pd.DataFrame`  
  and return a `pd.DataFrame` containing `time` and `metallicity`.

- **transient_name** : `str`  
  Name used to store the transient population in the population file.

- **oneline_cols** : `list[str]`, optional  
  Columns to extract from the oneline dataframe.  
  Default: all columns.

- **hist_cols** : `list[str]`, optional  
  Columns to extract from the history dataframe.  
  Default: all columns.

---

#### Returns
- **TransientPopulation** or **None**  
  A `TransientPopulation` object for interfacing with the transient population, or `None` if no systems are present.

---

#### Raises
- **ValueError**  
  - If the transient population is missing the `time` column.  
  - If the transient population contains duplicate columns.

---

#### Examples
See the tutorials for detailed usage examples.


___
Let’s create a transient population by keeping only the entries where `step_names = step_SN`. From these, we will retain the `remnant masses`, the `compact object type` (BH, NS, or WD), the `post-core-collapse binary state`, and the `times` of the `CC1` and `CC2` events.

In [ ]:
# Let's see the example again
pop.history[4][col]

## Building a Custom Function

In [ ]:
def postCC_lines(history_chunk, oneline_chunk, formation_channels_chunk=None):

    # here we identify the lines for each binary system when and if the experience collpase for star1 and star2.
    post_CC1 = ((history_chunk['step_names'] == "step_SN") &
                       (history_chunk['event'].shift(1) == "CC1"))  # Identifying only postCC1 lines
    post_CC2 = ((history_chunk['step_names'] == "step_SN") &
                       (history_chunk['event'].shift(1) == "CC2"))

    #So here we keep track of the compact_object formation (WD, NS, BH, or massless remnant) and the mass of the compact object
    #but note you can keep any other info that you are intersted. We will see more examples below. 
    # For CC1
    df1 = history_chunk.loc[post_CC1, ["time", "state", "S1_state", "S1_mass"]].copy()
    df1.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_state': 'stellar_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_mass': 'compact_object_mass'}, inplace=True)
    df1["metallicity"] = 1.0  # fixed value
    df1["progenitor_star"] = 1  # star 1 ended its life

    # For CC2
    df2 = history_chunk.loc[post_CC2, ["time", "state", "S2_state", "S2_mass"]].copy()
    df2.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_state': 'stellar_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_mass': "compact_object_mass"}, inplace=True)
    df2["metallicity"] = 1.0  # fixed value
    df2["progenitor_star"] = 2  # star 2 ended its life

    # Concatenate CC1 and CC2 vertically (rows stacked)
    df_synthetic = pd.concat([df1, df2], axis=0)

    return df_synthetic


## Create your transient_population

In [ ]:
postCC_pop = pop.create_transient_population(postCC_lines, 'temp')

## Printing out the outcome of the transient population we just created!

Let’s display the outcome of the transient population we just created! The table below summarizes the state of the binary after the collapse of the primary and secondary stars, along with the explosion times of both events. It also shows whether the system was disrupted or remained bound following each core-collapse, and whether the resulting remnant originated from a merger product or from a single star. In addition, it identifies the type of compact object formed and its mass (WD, NS, BH, or a massless remnant in the case of a PISN at low metallicity). In the progenitor column, a value of 1 refers to the primary star, while 2 refers to the secondary (if present).

In [ ]:
#col_CC = ["time", "binary_state_postCC", "stellar_state_postCC", "compact_object_mass", "progenitor_star"]
postCC_pop.population[:9]

## Example: Identify all NS and BH remnants formed from secondary stars in the transient population.

In [ ]:
# filter only NS and BH from progenitor star 2
mask = (postCC_pop.population["stellar_state_postCC"].isin(["NS", "BH"])) & (postCC_pop.population["progenitor_star"] == 2)
ns_bh_from_star2 = postCC_pop.population[mask]

# count NS and BH separately
count_ns = (ns_bh_from_star2["stellar_state_postCC"] == "NS").sum()
count_bh = (ns_bh_from_star2["stellar_state_postCC"] == "BH").sum()

print("From progenitor star 2:")
print("NS:", count_ns)
print("BH:", count_bh)


<div class="alert alert-success">

## Exercise: Mass distribution of the Black Holes

1. Find all black holes from initially single stars. (Replace the **XX** values on the code below)
2. Plot the BH mass distribution
   
</div>

In [ ]:
# STEP 1

mask_singles_BH = (XX) & (postCC_pop.population["stellar_state_postCC"].isin(["BH"]))

bh_from_singles = postCC_pop.population[mask_singles_BH]

# STEP 2
# write some code below to plot the BH mass distribution of your black holes

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint (click to reveal):</summary></b>

Find all the lines from postCC_pop.population where the "binary_state_postCC" column is equal to 'initially_single_star' & "stellar_state_postCC" is BH
    
</details>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>

```python
#STEP 1

mask_singles_BH = (postCC_pop.population["binary_state_postCC"].isin(["initially_single_star"])) & (postCC_pop.population["stellar_state_postCC"].isin(["BH"]))

bh_from_singles = postCC_pop.population[mask_singles_BH]

#STEP 2

bh_masses = bh_from_singles["compact_object_mass"]
plt.figure()
plt.hist(bh_masses, bins=10)
plt.xlabel(r"BH mass [$M_\odot$]")
plt.ylabel("Count")
plt.title("BH mass distribution (initially single stars)")
plt.show()


```
    
</details>

## 2. Introducing `key=oneline` data frame of the population

where `channel` , `SN_type`, `h1_mass_ej` & `he4_mass_ej` (total hydrogen and helium envelope mass in the ejecta, respectively) are stored.

## Printing out the columns saved in the oneline data frame of the population

In [ ]:
print(pop.oneline.columns)

The oneline DataFrame retains only the initial (*_i) and final (*_f) values of the binary system and its two stars, as well as values at specific evolutionary points (e.g., at core He depletion or core C ignition), which are needed for certain SN explodability prescriptions. As the name suggests, oneline contains a single row per binary, including initial and final conditions along with additional variables such as SN_type.

For more information, see: https://posydon.org/POSYDON/latest/tutorials-examples/population-synthesis/10_binaries_pop_syn.html#Population.oneline

<div class='alert alert-info'>
    
**Important**: When creating your own population runs, ensure that the output columns required in the .ini file are properly specified. Many columns in both the history DataFrame and oneline DataFrame are not saved by default. For the history DataFrame, set the only_select_columns to include the desired attributes of the binary, star_1, and star_2 objects. For the oneline DataFrame, make sure the relevant scalar_names for these objects are included.
    
</div>


# 2.1 Distinguishing ECSNe from CCSNe

## The column `SN_type`, stored under the `key=oneline`, contains information about the mechanism of supernova event (ECSN, PISN, PPI, or iron core collapse) depending on the SN prescription used.



<div class='alert alert-info'>

**Note:** This is the mechanism of explosion, NOT the observational type
    
</div>

## Modifying our custom function to include the SN_type information stored in the oneline DataFrame.

In [ ]:
def postCC_lines2(history_chunk, oneline_chunk, formation_channels_chunk=None):
    post_CC1 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC1"))
    post_CC2 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC2"))

    # CC1
    df1 = history_chunk.loc[post_CC1, ["time", "state", "S1_state", "S1_mass"]].copy()
    df1.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_state': 'stellar_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_mass': 'compact_object_mass'}, inplace=True)
    df1["metallicity"] = 1.0  # fixed value
    df1["progenitor_star"] = 1  # star 1 ended its life

    # For CC2
    df2 = history_chunk.loc[post_CC2, ["time", "state", "S2_state", "S2_mass"]].copy()
    df2.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_state': 'stellar_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_mass': "compact_object_mass"}, inplace=True)
    df2["metallicity"] = 1.0  # fixed value
    df2["progenitor_star"] = 2  # star 2 ended its life

    # Getting info for the same systems from oneline too
    
    
    idx1 = df1.index # The indeces in history to be used for the oneline
    idx2 = df2.index 
    
    df1["SN_type"] = oneline_chunk.loc[idx1, "S1_SN_type"]
    df2["SN_type"] = oneline_chunk.loc[idx2, "S2_SN_type"]

    # CC2 next to CC1 in columns
    return pd.concat([df1, df2], axis=0)


In [ ]:
postCC_pop = pop.create_transient_population(postCC_lines2, 'temp')

## Printing out the outcome of our transient population, now including the SN type information.

In [ ]:
postCC_pop.population[:10]

## Filter the transient population to show systems where star 1 or 2 produces an ECSN.

In [ ]:
ecsn_pop = postCC_pop.population[postCC_pop.population["SN_type"] == "ECSN"]
ecsn_pop.tail(5)

<div class="alert alert-success">

## Exercise: 
**Calculate the fraction of neutron stars (NS) originating from ECSNe versus those from CCSNe, relative to the total NS population, and then visualize these fractions using a pie chart. (Replace the **XX** values on the code below)**
</div>

In [ ]:
# Count CCSNe
ALL_CCSN = (
    ( XX? ).sum() 
)

# Count ECSNe
ALL_ECSN = (
    ( XX? ).sum()
)

print("The number of all CCSNe is", ALL_CCSN)
print("The number of all ECSNe is", ALL_ECSN)


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>

```python
ALL_ECSN = ((postCC_pop.population["stellar_state_postCC"].isin(["NS"])) & (postCC_pop.population["SN_type"] == "ECSN")).sum()
ALL_CCSN = ((postCC_pop.population["stellar_state_postCC"].isin(["NS"])) & (postCC_pop.population["SN_type"] == "CCSN")).sum()

```
    
</details>

## After estimating the number of NS formed from CCSNe and ECSNe, visualize the results in a pie chart using the following code:

In [ ]:
labels = ['CCSN', 'ECSN']
sizes = [ALL_CCSN, ALL_ECSN]
colors = ['grey', 'green']



plt.figure(figsize=(1.5, 1.5))
wedges, texts, autotexts = plt.pie(
    sizes,
    labels=labels,
    colors=colors,
    autopct=lambda p: f"{p:.1f}%" if p > 0 else "",
    startangle=90,
    counterclock=False,
    wedgeprops={"edgecolor": "black", "linewidth": 0.5}
)


for autotext in autotexts:
    autotext.set_fontsize(7)
    autotext.set_color("black")

plt.axis('equal')  # Keep it circular
plt.title("Collapse mechanism", fontsize=8)
plt.tight_layout()
plt.show()

<div class="alert alert-success">

## Exercise: 
**Plot the initial mass distribution of single stars compared to the mass distribution of primary stars in binary systems that produce ECSNe, modifying the custom function as needed to retain the initial masses.**
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint 1 (click to reveal):</summary></b>

```python

  #add the following code to the postCC_lines2 function 

    #Identify the initial conditions
    first_row= (history_chunk["step_names"] == "initial_cond")  

    
    df11=history_chunk.loc[first_row, ["S1_mass"]]
    df11.rename(columns={'initial_mass': 'S1_mass'}, inplace=True)

    
   
    df1=pd.merge(df1, df11, left_index=True, right_index=True, how='inner')


    #Then create the new transient population:

    postCC_pop = pop.create_transient_population(postCC_lines2, 'temp')

```
    
</details>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint 2 (click to reveal):</summary></b>

```python
#Plot the mass distribution of ECSNe from single and primary stars (normalized counts)


   
ECSN_from_binaries = (
    (postCC_pop.population["stellar_state_postCC"].isin(["NS"])) &
    (postCC_pop.population["SN_type"] == "ECSN") &
    (postCC_pop.population["binary_state_postCC"] != "initially_single_star")
)

ECSN_from_singles = (
    (postCC_pop.population["stellar_state_postCC"].isin(["NS"])) &
    (postCC_pop.population["SN_type"] == "ECSN") &
    (postCC_pop.population["binary_state_postCC"] == "initially_single_star")
)

# Extract initial masses
masses_singles = postCC_pop.population.loc[ECSN_from_singles, "initial_mass"].values
masses_binaries = postCC_pop.population.loc[ECSN_from_binaries, "initial_mass"].values

```
    
</details>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint 3 (click to reveal):</summary></b>

```python
# Find the initial masses of ECSNe arising from single and primary stars


   
# Combine datasets
all_masses = np.concatenate([masses_singles, masses_binaries])

# Remove NaNs or infinities
all_masses = all_masses[np.isfinite(all_masses)]

# Check if there’s enough range to create bins
if all_masses.min() == all_masses.max():
    # If all values are the same, create a single bin
    bins = [all_masses.min() - 0.25, all_masses.max() + 0.25]
else:
    # Define fixed-width bins
    bin_width = 0.5
    bins = np.arange(np.floor(all_masses.min()), np.ceil(all_masses.max()) + bin_width, bin_width)

# Plot
plt.figure(figsize=(8,5))
plt.hist(masses_singles, bins=bins, alpha=0.6, label="Single Stars",
         color='skyblue', edgecolor='black', density=True)
plt.hist(masses_binaries, bins=bins, alpha=0.6, label="Binary Stars",
         color='salmon', edgecolor='black', density=True)

plt.xlabel("$M_{1,initial}$ $(M_{\odot}$)")
plt.ylabel("Normalized Count")
plt.legend()
plt.show()

```
    
</details>

# 3. Observational SN types

## Classification of Observed SN Types

We separate **H-poor** (Ib, Ic, IIb) from **H-rich** (II) supernovae using hydrogen ejecta mass and surface nitrogen and helium abundances. 
To classify pre-CC models as progenitors of SNe II, IIb, Ib, or Ic, we use the total H mass in the ejecta (𝑀H,ej) as a primary indicator, along with the pre-SN surface He4 and N14 abundances evaluated at the time of core carbon depletion. However, the relationship between the structure of the pre-SN progenitor and the spectroscopic characteristics of the explosion remains an active area of research. Here we will just use some criteria that are suggested in the literature:


### Assumed Criteria based on literature 
(Gilkis et al. 2019, Yoon et al. 2017; Sravan et al. 2018, Aguilera-Dena et al. 2023, Dessart et al. 2020)
- **Ic:** $~M_{H,ej}$ < 0.033 $M_{\odot}$ $~\&~$  $X_{N,surf}$ < 1e-4 & $~\&~$  $X_{He4, surf}$ < 0.5
- **Ib:** $~M_{H,ej}$ < 0.033 $M_{\odot}$ $~\&~$  $X_{N,surf}$ >= 1e-4 & $~\&~$  $X_{He4, surf}$ >= 0.5
- **IIb:**  0.033 $M_{\odot}$ < $M_{H,ej}$ <= 0.5 $M_{\odot}$
- **II:** $~M_{H,ej}$ > 0.5 $M_{\odot}$

### Color Convention
- <span style="color:cyan; font-weight:bold">Type Ic</span> — cyan 
- <span style="color:blue; font-weight:bold">Type Ib</span> — blue  
- <span style="color:gold; font-weight:bold">Type IIb</span> — yellow  
- <span style="color:red; font-weight:bold">Type II</span> — red  



<div class='alert alert-info'>

Below is a function we created to classify each SN event as Ic, Ib, IIb or II. Please review it carefully to understand the parameters and the columns used. This time, we also need to retain the surface nitrogen and helium abundances at the time of core carbon depletion as well as the hydrogen mass ejecta during SNe,  and we have to modify the custom function accordingly.

</div>

In [ ]:
def classify_observed_SN(
    df,
    M_H_Ib=0.033,      # [Msun] Hydrogen ejecta threshold between Ib/Ic and IIb (Gilkis+2019)
    M_H_II=0.5,        # [Msun] Hydrogen ejecta threshold between IIb and II (Yoon+2017; Sravan+2018)
    N_surf_Ic=1e-4,    # [-] Surface nitrogen threshold to distinguish Ic from Ib (Aguilera-Dena+2023)
    he4_surf_Ic= 0.5,   # [-] Surface nitrogen threshold to distinguish Ic from Ib (Aguilera-Dena+2023)
    m_col="h1_mass_ej",  # Column containing hydrogen ejecta mass
    n_col="surface_n14",  # Column containing surface nitrogen abundance
    he4_col="surface_he4",  # Column containing surface helium abundance
    state="stellar_state_postCC"  # Column containing the state of the star after CC1 or CC2

):
    """
    

    The classification is based on the hydrogen ejecta mass (M) and surface 
    nitrogen abundance (N), surface Helium abundance (He4) using thresholds from the literature.

    Rules:
      - Type Ic :  M < M_H_Ib  and  N <  N_surf_Ic and h < he4_surf_Ic
      - Type Ib :  M < M_H_Ib  and  N >= N_surf_Ic and h >= he4_surf_Ic
      - Type IIb:  M_H_Ib < M < M_H_II
      - Type II :  M >= M_H_II
      - Otherwise: "Unknown"

    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe, one row per progenitor.
    M_H_Ib : float
        Threshold hydrogen ejecta mass (Msun) separating Ib/Ic from IIb.
    M_H_II : float
        Threshold hydrogen ejecta mass (Msun) separating IIb from II.
    N_surf_Ic : float
        Threshold surface nitrogen abundance distinguishing Ic from Ib.
    m_col : str
        Name of the column in df containing hydrogen ejecta mass.
    n_col : str
        Name of the column in df containing surface nitrogen abundance.

    Returns
    -------
    df : pandas.DataFrame
        Dataframe with one new column:
          - "SN_observed": the SN subtype (II, IIb, Ib, Ic, or Unknown).
    """

    # --- Check that required columns exist ---
    if m_col not in df.columns or n_col not in df.columns:
        raise KeyError(f"Expected columns '{m_col}' and '{n_col}' in df.")

    # Extract relevant quantities
    M = df[m_col]   # hydrogen ejecta mass
    N = df[n_col]   # surface nitrogen abundance
    h = df[he4_col] # surface helium abundance
    s= df[state] # post-CC state

    # Start with everything labeled as Unknown
    observed_type = df.index.to_series().map(lambda _: "Unknown")

    # Apply classification rules
    is_Ic  = (M < M_H_Ib) & (N <  N_surf_Ic) & (h <  he4_surf_Ic) & (s=="NS")
    is_Ib  = (M < M_H_Ib) & ((N >= N_surf_Ic) | (h >=  he4_surf_Ic)) & (s=="NS")
    is_IIb = (M > M_H_Ib) & (M <  M_H_II) & (s=="NS")
    is_II  = (M >= M_H_II) & (s=="NS")

    observed_type = observed_type.mask(is_Ic,  "Ic")
    observed_type = observed_type.mask(is_Ib,  "Ib")
    observed_type = observed_type.mask(is_IIb, "IIb")
    observed_type = observed_type.mask(is_II,  "II")

    # Save results back into the dataframe
    df["SN_observed"] = observed_type

    return df


<div class="alert alert-success">

## Exercise: Find the relative rates of all SN Types in the population

**1. Based on the assumed classification criteria above and the inputs used in the `classify_observed_SN`, determine which properties and columns need to be retained.**

**2. Replace the XX values in the `pre_and_postCC_lines` function below and create the new transient population keeping the information that should be used in the `classify_observed_SN` function.**
   
</div>

In [ ]:
def pre_and_postCC_lines(history_chunk, oneline_chunk, formation_channels_chunk=None):

    # NEW: collect the pre-SN lines at CC1/CC2, for surface abundances
    pre_CC1 = (history_chunk['event'] == "CC1") 
    pre_CC2 = (history_chunk['event'] == "CC2")  
    
    post_CC1 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC1"))
    post_CC2 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC2"))

    # CC1
    df1 = history_chunk.loc[post_CC1, ["time", "state", "S1_state", "S1_mass"]].copy()
    df1.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_state': 'stellar_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_mass': 'compact_object_mass'}, inplace=True)
    df1["metallicity"] = 1.0  # fixed value
    df1["progenitor_star"] = 1  # star 1 ended its life

    df1_pre = history_chunk.loc[pre_CC1, ['S1_surface_n14', XX? ]]  # ?
    df1_pre.rename(columns={'S1_surface_n14': 'surface_n14'}, inplace=True)
    df1_pre.rename(columns={'S1_surface_he4': 'surface_he4'}, inplace=True)

    
    df1 = pd.merge(df1, df1_pre, left_index=True, right_index=True, how='inner')

    # For CC2
    df2 = history_chunk.loc[post_CC2, ["time", "state", "S2_state", "S2_mass"]].copy()
    df2.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_state': 'stellar_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_mass': "compact_object_mass"}, inplace=True)
    df2["metallicity"] = 1.0  # fixed value
    df2["progenitor_star"] = 2  # star 2 ended its life

    df2_pre = history_chunk.loc[pre_CC2, ['S2_surface_n14', XX? ]]   # ?
    df2_pre.rename(columns={'S2_surface_n14': 'surface_n14'}, inplace=True)
    df2_pre.rename(columns={'S2_surface_he4': 'surface_he4'}, inplace=True)

    df2 = pd.merge(df2, df2_pre, left_index=True, right_index=True, how='inner')

    # Getting info for the same systems from oneline too

    idx1 = df1.index # The indeces in history to be used for the oneline
    idx2 = df2.index 
    
    df1["SN_type"] = oneline_chunk.loc[idx1, "S1_SN_type"]
    df2["SN_type"] = oneline_chunk.loc[idx2, "S2_SN_type"]

    df1["h1_mass_ej"] = oneline_chunk.loc[idx1, XX? ] # ?
    df2["h1_mass_ej"] = oneline_chunk.loc[idx2, XX? ] # ?

    # CC2 next to CC1 in columns
    return pd.concat([df1, df2], axis=0)


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint 1 (click to reveal):</summary></b>
Some properties/columns describe the preCC (progenitor) state, others pertain to the compact object, and a few are listed directly in the outline since they were pre-calculated from the profile.
</details>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint 2 (click to reveal):</summary></b>
We should modify the custom function to include information of the surface abundances of the stars 
before SNe (at core carbon depletion) adding the appropriate columns.

- **preSN** line → progenitor properties (e.g., surface abundances, core mass etc). In our case we need `surface_he4`,`surface_n14`, preSN mass of the progenitor
- **postSN** line → compact object mass and state, orbit re-adjustment after instataneous mass loss and natal kick, etc. In our case we need compact object mass and state
- **oneline** dataframe → keeping some usually pre_computed info about the event. In our case we need `h1_mass_ej`

</details>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>

```python
   
def pre_and_postCC_lines(history_chunk, oneline_chunk, formation_channels_chunk=None):

    # NEW: collect the pre-SN lines at CC1/CC2, for surface abundances
    pre_CC1 = (history_chunk['event'] == "CC1") 
    pre_CC2 = (history_chunk['event'] == "CC2")  
    
    post_CC1 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC1"))
    post_CC2 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC2"))

    # CC1
    df1 = history_chunk.loc[post_CC1, ["time", "state", "S1_state", "S1_mass"]].copy()
    df1.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_state': 'stellar_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_mass': 'compact_object_mass'}, inplace=True)
    df1["metallicity"] = 1.0  # fixed value
    df1["progenitor_star"] = 1  # star 1 ended its life

    df1_pre = history_chunk.loc[pre_CC1, ['S1_surface_n14', 'S1_surface_he4' ]] 
    df1_pre.rename(columns={'S1_surface_n14': 'surface_n14'}, inplace=True)
    df1_pre.rename(columns={'S1_surface_he4': 'surface_he4'}, inplace=True)

    
    df1 = pd.merge(df1, df1_pre, left_index=True, right_index=True, how='inner')

    # For CC2
    df2 = history_chunk.loc[post_CC2, ["time", "state", "S2_state", "S2_mass"]].copy()
    df2.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_state': 'stellar_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_mass': "compact_object_mass"}, inplace=True)
    df2["metallicity"] = 1.0  # fixed value
    df2["progenitor_star"] = 2  # star 2 ended its life

    df2_pre = history_chunk.loc[pre_CC2, ['S2_surface_n14','S2_surface_he4' ]]  
    df2_pre.rename(columns={'S2_surface_n14': 'surface_n14'}, inplace=True)
    df2_pre.rename(columns={'S2_surface_he4': 'surface_he4'}, inplace=True)

    df2 = pd.merge(df2, df2_pre, left_index=True, right_index=True, how='inner')

    # Getting info for the same systems from oneline too

    idx1 = df1.index # The indeces in history to be used for the oneline
    idx2 = df2.index 
    
    df1["SN_type"] = oneline_chunk.loc[idx1, "S1_SN_type"]
    df2["SN_type"] = oneline_chunk.loc[idx2, "S2_SN_type"]

    df1["h1_mass_ej"] = oneline_chunk.loc[idx1, "S1_h1_mass_ej"] 
    df2["h1_mass_ej"] = oneline_chunk.loc[idx2, "S2_h1_mass_ej"] 

    # CC2 next to CC1 in columns
    return pd.concat([df1, df2], axis=0)

```
    
</details>

In [ ]:
CC_pop = pop.create_transient_population(pre_and_postCC_lines, 'temp')

In [ ]:
CC_pop.population[:8]

Estimate the relative rates of different observational supernova (SN) types. Absolute rates would require normalization by the mass formed in the stellar populations (see this afternoon’s lecture for details).

In [ ]:
# Classify observed SN types with your chosen thresholds
df_classified = classify_observed_SN(CC_pop.population, M_H_Ib=0.033, M_H_II=0.5, N_surf_Ic=1e-4)

# Collect observed SN types for both stars
obs = df_classified["SN_observed"]
obs = obs[obs.isin(["Ic", "Ib", "IIb", "II"])]  # drop Unknown if present

# Count and normalize
counts = obs.value_counts().reindex(["Ic", "Ib", "IIb", "II"], fill_value=0)
fractions = counts.values.astype(float)
fractions = fractions / fractions.sum()

# Labels and colors
labels = ["Type Ic", "Type Ib", "Type IIb", "Type II"]
colors = ["cyan", "blue", "yellow", "red"]
explode = [0.05] * 4

# Plot pie chart
plt.figure(figsize=(2.0, 2.0))
wedges, texts, autotexts = plt.pie(
    fractions,
    labels=labels,
    colors=colors,
    explode=explode,
    autopct=lambda p: f"{p:.1f}%" if p > 0 else "",
    startangle=90,
    counterclock=False,
    wedgeprops={"edgecolor": "black", "linewidth": 0.5}
)

for autotext in autotexts:
    autotext.set_fontsize(7)
    autotext.set_color("black")

plt.title("Fraction of Observed Supernova Types")
plt.show()


# 4. Total ejecta masses

<div class="alert alert-success">


## Exercise: Calculate the distribution of ejecta masses of SNe. 
Let's stick to **core-collapse SNe**  and to events that form **neutron stars** only, to not complicate it with potential fallback from black holes and for Observed **Type Ib** SNe only. Fill in the ## placeholders in the code below.

   
</div>

First we will keep the information of preSN mass of the stellar progenitor

In [ ]:
def ejecta_mass(history_chunk, oneline_chunk, formation_channels_chunk=None):

    pre_CC1 =  # ?
    pre_CC2 =  # ?
    
    post_CC1 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC1"))
    post_CC2 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC2"))

    # CC1
    df1 = history_chunk.loc[post_CC1, ["time", "state", "S1_state", "S1_mass"]].copy()
    df1.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_state': 'stellar_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_mass': 'compact_object_mass'}, inplace=True)
    df1["metallicity"] = 1.0  # fixed value
    df1["progenitor_star"] = 1  # star 1 ended its life

    df1_pre = history_chunk.loc[pre_CC1, ['S1_surface_n14', 'S1_surface_he4' , ##?]]  #  also grab preSN mass # ?
    df1_pre.rename(columns={##? : 'preCC_mass'}, inplace=True)   # rename clearly  # ?
    df1_pre.rename(columns={'S1_surface_n14': 'surface_n14'}, inplace=True)
    df1_pre.rename(columns={'S1_surface_he4': 'surface_he4'}, inplace=True)

    
    df1 = pd.merge(df1, df1_pre, left_index=True, right_index=True, how='inner')

    # For CC2
    df2 = history_chunk.loc[post_CC2, ["time", "state", "S2_state", "S2_mass"]].copy()
    df2.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_state': 'stellar_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_mass': "compact_object_mass"}, inplace=True)
    df2["metallicity"] = 1.0  # fixed value
    df2["progenitor_star"] = 2  # star 2 ended its life

    df2_pre = history_chunk.loc[pre_CC2, ['S2_surface_n14','S2_surface_he4', ##?]] 
    df2_pre.rename(columns={##? : 'preCC_mass'}, inplace=True)   
    df2_pre.rename(columns={'S2_surface_n14': 'surface_n14'}, inplace=True) 
    df2_pre.rename(columns={'S2_surface_he4': 'surface_he4'}, inplace=True) 

    
    df2 = pd.merge(df2, df2_pre, left_index=True, right_index=True, how='inner')

    # Getting info for the same systems from oneline too

    idx1 = df1.index # The indeces in history to be used for the oneline
    idx2 = df2.index 
    
    df1["SN_type"] = oneline_chunk.loc[idx1, "S1_SN_type"]
    df2["SN_type"] = oneline_chunk.loc[idx2, "S2_SN_type"]

    df1["h1_mass_ej"] = oneline_chunk.loc[idx1, "S1_h1_mass_ej"]
    df2["h1_mass_ej"] = oneline_chunk.loc[idx2, "S2_h1_mass_ej"] 

    # CC2 next to CC1 in columns
    return pd.concat([df1, df2], axis=0)


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint (click to reveal):</summary></b>

The ejecta mass is given by  

$
M_{\mathrm{ej}} = M_{\mathrm{prog}} - M_{\mathrm{rem}}
\$

where $(M_{\mathrm{prog}})$ is the progenitor mass at core carbon depletion, and $(M_{\mathrm{rem}})$ is the mass of the neutron star.  

</details>


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>
```

def ejecta_mass(history_chunk, oneline_chunk, formation_channels_chunk=None):

    pre_CC1 = (history_chunk['event'] == "CC1") 
    pre_CC2 = (history_chunk['event'] == "CC2") 
    
    post_CC1 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC1"))
    post_CC2 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC2"))

    # CC1
    df1 = history_chunk.loc[post_CC1, ["time", "state", "S1_state", "S1_mass"]].copy()
    df1.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_state': 'stellar_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_mass': 'compact_object_mass'}, inplace=True)
    df1["metallicity"] = 1.0  # fixed value
    df1["progenitor_star"] = 1  # star 1 ended its life

    df1_pre = history_chunk.loc[pre_CC1, ['S1_surface_n14', 'S1_surface_he4' , 'S1_mass']]  
    df1_pre.rename(columns={'S1_mass' : 'preCC_mass'}, inplace=True)   
    df1_pre.rename(columns={'S1_surface_n14': 'surface_n14'}, inplace=True)
    df1_pre.rename(columns={'S1_surface_he4': 'surface_he4'}, inplace=True)

    
    df1 = pd.merge(df1, df1_pre, left_index=True, right_index=True, how='inner')

    # For CC2
    df2 = history_chunk.loc[post_CC2, ["time", "state", "S2_state", "S2_mass"]].copy()
    df2.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_state': 'stellar_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_mass': "compact_object_mass"}, inplace=True)
    df2["metallicity"] = 1.0  # fixed value
    df2["progenitor_star"] = 2  # star 2 ended its life

    df2_pre = history_chunk.loc[pre_CC2, ['S2_surface_n14','S2_surface_he4', 'S2_mass']] 
    df2_pre.rename(columns={'S2_mass' : 'preCC_mass'}, inplace=True)   
    df2_pre.rename(columns={'S2_surface_n14': 'surface_n14'}, inplace=True) 
    df2_pre.rename(columns={'S2_surface_he4': 'surface_he4'}, inplace=True) 

   # df2["mass_ejecta"]=df2_pre['preCC_mass']-df2['compact_object_mass']


    
    df2 = pd.merge(df2, df2_pre, left_index=True, right_index=True, how='inner')


    idx1 = df1.index # The indeces in history to be used for the oneline
    idx2 = df2.index 
    
    df1["SN_type"] = oneline_chunk.loc[idx1, "S1_SN_type"]
    df2["SN_type"] = oneline_chunk.loc[idx2, "S2_SN_type"]

    df1["h1_mass_ej"] = oneline_chunk.loc[idx1, "S1_h1_mass_ej"] 
    df2["h1_mass_ej"] = oneline_chunk.loc[idx2, "S2_h1_mass_ej"] 

    

    # CC2 next to CC1 in columns
    return pd.concat([df1, df2], axis=0)

```
</details>

In [ ]:
Mejecta_pop = pop.create_transient_population(ejecta_mass, 'temp')

In [ ]:
Mejecta_pop.population

## Estimating the ejecta masses of CC1 and CC2 for type Ib SNe



In [ ]:
# Run classification on the underlying DataFrame
df_classified = classify_observed_SN(
    Mejecta_pop.population,
    M_H_Ib=0.033, M_H_II=0.5, N_surf_Ic=1e-4
)

# Compute ejecta mass = preSN mass - remnant mass  # CHANGED: explicitly add ejecta columns
df_classified["M_ejecta"] = df_classified["preCC_mass"] - df_classified["compact_object_mass"]

# Identify NS remnants via compact-object mass window [M_sun]
is_NS = df_classified['compact_object_mass'].between(1.0, 3.0, inclusive='both')

# Observed Type Ib masks
is_Ib = df_classified['SN_observed'].eq('Ib')

# Ejecta masses for Ib + NS only, positive values
M_ej_Ib_NS = pd.concat([
    df_classified.loc[is_NS & is_Ib, 'M_ejecta'],
]).dropna()
M_ej_Ib_NS = M_ej_Ib_NS[M_ej_Ib_NS > 0]

# Plot
plt.figure(figsize=(2,2))
plt.hist(M_ej_Ib_NS, bins=15, alpha=0.85)
plt.xlabel(r"$M_{\mathrm{ej}}$ [M$_\odot$]")
plt.ylabel("Count")
plt.title("Ejecta masses: Type Ib CCSNe with NS remnants")
plt.tight_layout()
plt.show()


# Inclusion of channels and find which are the most dominant ones contributing to Type Ib.

This section covers how to obtain information about the evolutionary history of binary systems prior to their collapse and display the Van den Heuvel diagram

In [ ]:
pop.calculate_formation_channels()

In [ ]:
def ejecta_mass_withchannels(history_chunk, oneline_chunk, formation_channels_chunk=None):

    pre_CC1 = (history_chunk['event'] == "CC1")  
    pre_CC2 = (history_chunk['event'] == "CC2")  
    
    post_CC1 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC1"))
    post_CC2 = ((history_chunk['step_names'] == "step_SN") & (history_chunk['event'].shift(1) == "CC2"))

    # CC1
    df1 = history_chunk.loc[post_CC1, ["time", "state", "S1_state", "S1_mass"]].copy()
    df1.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_state': 'stellar_state_postCC'}, inplace=True)
    df1.rename(columns={'S1_mass': 'compact_object_mass'}, inplace=True)
    df1["metallicity"] = 1.0  # fixed value
    df1["progenitor_star"] = 1  # star 1 ended its life

    df1_pre = history_chunk.loc[pre_CC1, ['S1_surface_n14', 'S1_surface_he4','S1_mass','event']]  #  also grab preSN mass # ?
    df1_pre.rename(columns={'S1_mass': 'preCC_mass'}, inplace=True)   # rename clearly  # ?s
    df1_pre.rename(columns={'S1_surface_n14': 'surface_n14'}, inplace=True) 
    df1_pre.rename(columns={'S1_surface_he4': 'surface_he4'}, inplace=True) 


    
    df1 = pd.merge(df1, df1_pre, left_index=True, right_index=True, how='inner')

    # For CC2
    df2 = history_chunk.loc[post_CC2, ["time", "state", "S2_state", "S2_mass"]].copy()
    df2.rename(columns={'state': 'binary_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_state': 'stellar_state_postCC'}, inplace=True)
    df2.rename(columns={'S2_mass': "compact_object_mass"}, inplace=True)
    df2["metallicity"] = 1.0  # fixed value
    df2["progenitor_star"] = 2  # star 2 ended its life

    df2_pre = history_chunk.loc[pre_CC2, ['S2_surface_n14','S2_surface_he4', 'S2_mass', 'event']]  #  also grab preSN mass # ?
    df2_pre.rename(columns={'S2_mass': 'preCC_mass'}, inplace=True)   
    df2_pre.rename(columns={'S2_surface_n14': 'surface_n14'}, inplace=True) 
    df2_pre.rename(columns={'S2_surface_he4': 'surface_he4'}, inplace=True) 

    
    df2 = pd.merge(df2, df2_pre, left_index=True, right_index=True, how='inner')

    # Getting info for the same systems from oneline too

    idx1 = df1.index # The indeces in history to be used for the oneline
    idx2 = df2.index 
    
    df1["SN_type"] = oneline_chunk.loc[idx1, "S1_SN_type"]
    df2["SN_type"] = oneline_chunk.loc[idx2, "S2_SN_type"]

    df1["h1_mass_ej"] = oneline_chunk.loc[idx1, "S1_h1_mass_ej"] 
    df2["h1_mass_ej"] = oneline_chunk.loc[idx2, "S2_h1_mass_ej"] 
    df2['channel'] = formation_channels_chunk.loc[idx2, "channel"]
    df1['channel'] = formation_channels_chunk.loc[idx1, 'channel']


    # CC2 next to CC1 in columns
    df_synthetic=pd.concat([df1, df2], axis=0)
    df_synthetic['binary_index']=df_synthetic.index
    return df_synthetic


In [ ]:
Mejecta_pop_with_channels = pop.create_transient_population(ejecta_mass_withchannels, 'temp')

<div class="alert alert-success">

# Excercise:
Identify the evolutionary channels of star 1 and star 2 that can lead to the production of Type Ib supernovae. Then, determine from the most dominant evolutionary path one binary parameter index and use it to plot the Van den Heuvel diagrams for this system.

</div>

In [ ]:
typeIb_SNe_from_star1 = Mejecta_pop_with_channels.population[
        (Mejecta_pop_with_channels.population['stellar_state_postCC'] == 'NS') & 
        (Mejecta_pop_with_channels.population['event'] == 'CC1') &
        (Mejecta_pop_with_channels.population['h1_mass_ej'] < 0.033) &
        ((Mejecta_pop_with_channels.population['surface_n14'] > 1e-4) |  (Mejecta_pop_with_channels.population['surface_he4'] > 0.5))
    ]

channels_star1 = (typeIb_SNe_from_star1['channel']).value_counts()
print(channels_star1)

In [ ]:
typeIb_SNe_from_star1 = Mejecta_pop_with_channels.population[
        (Mejecta_pop_with_channels.population['stellar_state_postCC'] == 'NS') & 
        (Mejecta_pop_with_channels.population['event'] == 'CC1') &
        (Mejecta_pop_with_channels.population['h1_mass_ej'] < 0.033) &
        ((Mejecta_pop_with_channels.population['surface_n14'] > 1e-4) |  (Mejecta_pop_with_channels.population['surface_he4'] > 0.5))&
        (Mejecta_pop_with_channels.population['channel'] == 'ZAMS_oRLO1_CC1_CC2_END')
    ]

channels_star1 = (typeIb_SNe_from_star1['binary_index']).value_counts()
print(channels_star1)

If you install POSYDON on your computer following the instructions provided in the linked guide, you’ll have the option to enable experimental visualization libraries. While these libraries offer advanced features, please note that they might still be in development and could be subject to changes.
Instructions for visualizations: https://posydon.org/POSYDON/latest/getting-started/installation-guide.html#id11

To install these experimental visualization libraries
Navigate to your POSYDON directory (where the setup.py is located) and run:
pip install ".[vis]"

Unfortunataly, this will not work in the quest environemnt for the School. 
See the example commands below. TAs will show an example diagram.

```python
from posydon.visualization.VHdiagram import VHdiagram
VHdiagram('1e+00_Zsun_population.h5', path='/projects/e33022/POSYDON-shared/populations/5k_Manos_Dimitris/', index=4)
```

```python

from posydon.visualization.VHdiagram import DisplayMode
from posydon.visualization.VH_diagram.Presenter import PresenterMode

VHdiagram(
    '1e+00_Zsun_population.h5', path='/projects/e33022/POSYDON-shared/populations/5k_Manos_Dimitris/',
    index=21,
    presentMode=PresenterMode.DIAGRAM,
    displayMode=DisplayMode.INLINE_B,
)

```

<div class='alert alert-info'>

If you’re efficient and have time left, repeat the process for secondaries to identify the most dominant channel producing type Ic SNe.

</div>